In [2]:
#import
import pandas as pd
import numpy as np

#PATH dove andare a leggere e a scrivere
PATH_TO_WRITE_ETL = "./dataset_modificato_WC/"
PATH_FROM_READ_ETL = "./dataset_originale_WC/"
PATH_FROM_READ_ANALYSIS = "./dataset_modificato_WC/" 

In [3]:
df_tournaments = pd.read_csv(PATH_FROM_READ_ETL + "/tournaments.csv")

df_etl = df_tournaments[['host_country']]
df_etl.at[16,'host_country']='Japan'
new_row = {'host_country':'Korea'}
df_etl = df_etl.append(new_row, ignore_index=True)
arr = np.ones(22)
df_etl['numero_volte'] = arr
df_etl = df_etl.groupby(['host_country'])['numero_volte'].sum().reset_index(name = 'numero_volte')

for i in df_etl.index:
    if(df_etl['numero_volte'][i] == 1): df_etl['numero_volte'][i] = '1'
    else: df_etl['numero_volte'][i] = '2'

df_etl.to_csv('./etl_Simone/country_tournaments.csv', index = False)

C:\Users\simod\AppData\Local\Temp\ipykernel_13260\2708306084.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_etl = df_etl.append(new_row, ignore_index=True)
C:\Users\simod\AppData\Local\Temp\ipykernel_13260\2708306084.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if(df_etl['numero_volte'][i] == 1): df_etl['numero_volte'][i] = '1'


In [17]:
df_gmap = pd.read_csv(PATH_TO_WRITE_ETL + "player_appearances.csv")
df_ts = pd.read_csv(PATH_TO_WRITE_ETL + "tournament_standings.csv")

df_ts = df_ts[['team_name','position']]
df_ts = df_ts.query('position == 1')
df_ts = df_ts.drop_duplicates().reset_index()
df_ts = df_ts.drop(['index'], axis=1)

df_gmap = df_gmap[['stage_name','team_name']]
df_gmap = df_gmap.drop_duplicates()
df_gmap.sort_values(by = 'team_name', ascending = True, inplace = True)
new_col = np.zeros(255)
df_gmap['number'] = new_col

for ind in df_gmap.index:
    if(df_gmap['stage_name'][ind] == 'group stage'): df_gmap['number'][ind] = 0
    elif(df_gmap['stage_name'][ind] == 'second group stage'): df_gmap['number'][ind] = 1
    elif(df_gmap['stage_name'][ind] == 'round of 16'): df_gmap['number'][ind] = 2
    elif(df_gmap['stage_name'][ind] == 'quarter-finals'): df_gmap['number'][ind] = 3
    elif(df_gmap['stage_name'][ind] == 'semi-finals'): df_gmap['number'][ind] = 4
    elif(df_gmap['stage_name'][ind] == 'third-place match'): df_gmap['number'][ind] = 5
    elif(df_gmap['stage_name'][ind] == 'final round'): df_gmap['number'][ind] = 6
    elif(df_gmap['stage_name'][ind] == 'final'): df_gmap['number'][ind] = 7

df_gmap = df_gmap.groupby(['team_name'])['number'].max().reset_index(name = 'number')

for ind in df_gmap.index:
    if(df_gmap['number'][ind] == 0): df_gmap['number'][ind] = 'group stage'
    elif(df_gmap['number'][ind] == 1): df_gmap['number'][ind] = 'second group stage'
    elif(df_gmap['number'][ind] == 2): df_gmap['number'][ind] = 'round of 16'
    elif(df_gmap['number'][ind] == 3): df_gmap['number'][ind] = 'quarter-finals'
    elif(df_gmap['number'][ind] == 4): df_gmap['number'][ind] = 'semi-finals'
    elif(df_gmap['number'][ind] == 5): df_gmap['number'][ind] = 'third-place match'
    elif(df_gmap['number'][ind] == 6): df_gmap['number'][ind] = 'final round'
    elif(df_gmap['number'][ind] == 7): df_gmap['number'][ind] = 'final'

df_gmap = df_gmap.rename(columns={"number": "stage_name"})

for i in df_ts.index:
    for j in df_gmap.index:
        if(df_ts['team_name'][i] == df_gmap['team_name'][j]):
            df_gmap['stage_name'][j] = 'champion'

df_gmap.loc[df_gmap.team_name == 'England','team_name'] = 'United Kingdom'

df_gmap.to_csv('./etl_Simone/team_stage.csv', index = False)

df_gmap

C:\Users\simod\AppData\Local\Temp\ipykernel_15264\4231784084.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if(df_gmap['stage_name'][ind] == 'group stage'): df_gmap['number'][ind] = 0
C:\Users\simod\AppData\Local\Temp\ipykernel_15264\4231784084.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elif(df_gmap['stage_name'][ind] == 'round of 16'): df_gmap['number'][ind] = 2
C:\Users\simod\AppData\Local\Temp\ipykernel_15264\4231784084.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

,team_name,stage_name
0,Algeria,round of 16
1,Angola,group stage
2,Argentina,champion
3,Australia,round of 16
4,Austria,third-place match
...,...,...
79,Uruguay,champion
80,Wales,quarter-finals
81,West Germany,champion
82,Yugoslavia,third-place match


In [70]:
df_gmap = pd.read_csv(PATH_TO_WRITE_ETL + "player_appearances.csv")
df_goals = pd.read_csv(PATH_TO_WRITE_ETL + "goals.csv")
df_ts = pd.read_csv(PATH_TO_WRITE_ETL + "tournament_standings.csv")

df_gmap = df_gmap[['tournament_id','stage_name','team_name']]
df_gmap = df_gmap.drop_duplicates().reset_index()
df_gmap = df_gmap.drop(['index'], axis=1)
df_gmap['goals'] = 0

df_goals = df_goals[['tournament_id','team_name']]
df_goals['goals'] = 1
df_goals = df_goals.groupby(['tournament_id','team_name'])['goals'].sum().reset_index(name = 'goals')

df_ts = df_ts[['tournament_id','team_name','position']]
df_ts = df_ts.query('position == 1').reset_index()
df_ts = df_ts.drop(['index'], axis=1)

for i in df_goals.index:
    for j in df_gmap.index:
        if(df_goals['tournament_id'][i] == df_gmap['tournament_id'][j] and df_goals['team_name'][i] == df_gmap['team_name'][j]):
            df_gmap['goals'][j] = df_goals['goals'][i]

df_gmap['number'] = 0

for ind in df_gmap.index:
    if(df_gmap['stage_name'][ind] == 'group stage'): df_gmap['number'][ind] = 0
    elif(df_gmap['stage_name'][ind] == 'second group stage'): df_gmap['number'][ind] = 1
    elif(df_gmap['stage_name'][ind] == 'round of 16'): df_gmap['number'][ind] = 2
    elif(df_gmap['stage_name'][ind] == 'quarter-finals'): df_gmap['number'][ind] = 3
    elif(df_gmap['stage_name'][ind] == 'semi-finals'): df_gmap['number'][ind] = 4
    elif(df_gmap['stage_name'][ind] == 'third-place match'): df_gmap['number'][ind] = 5
    elif(df_gmap['stage_name'][ind] == 'final round'): df_gmap['number'][ind] = 6
    elif(df_gmap['stage_name'][ind] == 'final'): df_gmap['number'][ind] = 7

df_gmap_1930 = df_gmap[df_gmap.tournament_id.eq('WC-1930')].reset_index()
df_gmap_1930 = df_gmap_1930.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_1934 = df_gmap[df_gmap.tournament_id.eq('WC-1934')].reset_index()
df_gmap_1934 = df_gmap_1934.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_1938 = df_gmap[df_gmap.tournament_id.eq('WC-1938')].reset_index()
df_gmap_1938 = df_gmap_1938.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_1950 = df_gmap[df_gmap.tournament_id.eq('WC-1950')].reset_index()
df_gmap_1950 = df_gmap_1950.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_1954 = df_gmap[df_gmap.tournament_id.eq('WC-1954')].reset_index()
df_gmap_1954 = df_gmap_1954.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_1958 = df_gmap[df_gmap.tournament_id.eq('WC-1958')].reset_index()
df_gmap_1958 = df_gmap_1958.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_1962 = df_gmap[df_gmap.tournament_id.eq('WC-1962')].reset_index()
df_gmap_1962 = df_gmap_1962.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_1966 = df_gmap[df_gmap.tournament_id.eq('WC-1966')].reset_index()
df_gmap_1966 = df_gmap_1966.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_1970 = df_gmap[df_gmap.tournament_id.eq('WC-1970')].reset_index()
df_gmap_1970 = df_gmap_1970.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_1974 = df_gmap[df_gmap.tournament_id.eq('WC-1974')].reset_index()
df_gmap_1974 = df_gmap_1974.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_1978 = df_gmap[df_gmap.tournament_id.eq('WC-1978')].reset_index()
df_gmap_1978 = df_gmap_1978.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_1982 = df_gmap[df_gmap.tournament_id.eq('WC-1982')].reset_index()
df_gmap_1982 = df_gmap_1982.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_1986 = df_gmap[df_gmap.tournament_id.eq('WC-1986')].reset_index()
df_gmap_1986 = df_gmap_1986.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_1990 = df_gmap[df_gmap.tournament_id.eq('WC-1990')].reset_index()
df_gmap_1990 = df_gmap_1990.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_1994 = df_gmap[df_gmap.tournament_id.eq('WC-1994')].reset_index()
df_gmap_1994 = df_gmap_1994.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_1998 = df_gmap[df_gmap.tournament_id.eq('WC-1998')].reset_index()
df_gmap_1998 = df_gmap_1998.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_2002 = df_gmap[df_gmap.tournament_id.eq('WC-2002')].reset_index()
df_gmap_2002 = df_gmap_2002.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_2006 = df_gmap[df_gmap.tournament_id.eq('WC-2006')].reset_index()
df_gmap_2006 = df_gmap_2006.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_2010 = df_gmap[df_gmap.tournament_id.eq('WC-2010')].reset_index()
df_gmap_2010 = df_gmap_2010.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_2014 = df_gmap[df_gmap.tournament_id.eq('WC-2014')].reset_index()
df_gmap_2014 = df_gmap_2014.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

df_gmap_2018 = df_gmap[df_gmap.tournament_id.eq('WC-2018')].reset_index()
df_gmap_2018 = df_gmap_2018.groupby(['tournament_id','team_name','goals'])['number'].max().reset_index(name = 'number')

dataframes = [df_gmap_1930, df_gmap_1934, df_gmap_1938, df_gmap_1950, df_gmap_1954, df_gmap_1958,
df_gmap_1962, df_gmap_1966, df_gmap_1970, df_gmap_1974, df_gmap_1978, df_gmap_1982, df_gmap_1986,
df_gmap_1990, df_gmap_1994, df_gmap_1998, df_gmap_2002, df_gmap_2006, df_gmap_2010, df_gmap_2014,
df_gmap_2018]

df_gmap = pd.concat(dataframes).reset_index()
df_gmap = df_gmap.drop(['index'], axis=1)

for ind in df_gmap.index:
    if(df_gmap['number'][ind] == 0): df_gmap['number'][ind] = 'group stage'
    elif(df_gmap['number'][ind] == 1): df_gmap['number'][ind] = 'second group stage'
    elif(df_gmap['number'][ind] == 2): df_gmap['number'][ind] = 'round of 16'
    elif(df_gmap['number'][ind] == 3): df_gmap['number'][ind] = 'quarter-finals'
    elif(df_gmap['number'][ind] == 4): df_gmap['number'][ind] = 'semi-finals'
    elif(df_gmap['number'][ind] == 5): df_gmap['number'][ind] = 'third-place match'
    elif(df_gmap['number'][ind] == 6): df_gmap['number'][ind] = 'final round'
    elif(df_gmap['number'][ind] == 7): df_gmap['number'][ind] = 'final'

df_gmap = df_gmap.rename(columns={"number": "stage_name"})

for i in df_ts.index:
    for j in df_gmap.index:
        if(df_ts['team_name'][i] == df_gmap['team_name'][j] and df_ts['tournament_id'][i] == df_gmap['tournament_id'][j]):
            df_gmap['stage_name'][j] = 'champion'

#df_gmap.to_csv('./etl_Simone/team_goals.csv', index = False)

df_gmap

C:\Users\simod\AppData\Local\Temp\ipykernel_15264\2000818993.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gmap['goals'][j] = df_goals['goals'][i]
C:\Users\simod\AppData\Local\Temp\ipykernel_15264\2000818993.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if(df_gmap['stage_name'][ind] == 'group stage'): df_gmap['number'][ind] = 0
C:\Users\simod\AppData\Local\Temp\ipykernel_15264\2000818993.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

,tournament_id,team_name,goals,stage_name
0,WC-1930,Argentina,18,final
1,WC-1930,Belgium,0,group stage
2,WC-1930,Bolivia,0,group stage
3,WC-1930,Brazil,5,group stage
4,WC-1930,Chile,5,group stage
...,...,...,...,...
452,WC-2018,Spain,7,round of 16
453,WC-2018,Sweden,6,quarter-finals
454,WC-2018,Switzerland,5,round of 16
455,WC-2018,Tunisia,5,group stage


In [5]:
df_team_goals = pd.read_csv("./etl_Simone/team_goals.csv")

df_team_goals['stage_number'] = 0

for ind in df_team_goals.index:
    if(df_team_goals['stage_name'][ind] == 'group stage'): df_team_goals['stage_number'][ind] = 0
    elif(df_team_goals['stage_name'][ind] == 'second group stage'): df_team_goals['stage_number'][ind] = 1
    elif(df_team_goals['stage_name'][ind] == 'round of 16'): df_team_goals['stage_number'][ind] = 2
    elif(df_team_goals['stage_name'][ind] == 'quarter-finals'): df_team_goals['stage_number'][ind] = 3
    elif(df_team_goals['stage_name'][ind] == 'semi-finals'): df_team_goals['stage_number'][ind] = 4
    elif(df_team_goals['stage_name'][ind] == 'third-place match'): df_team_goals['stage_number'][ind] = 5
    elif(df_team_goals['stage_name'][ind] == 'final round'): df_team_goals['stage_number'][ind] = 6
    elif(df_team_goals['stage_name'][ind] == 'final'): df_team_goals['stage_number'][ind] = 7
    elif(df_team_goals['stage_name'][ind] == 'champion'): df_team_goals['stage_number'][ind] = 8

#df_team_goals.to_csv('./etl_Simone/team_goals.csv', index = False)

df_team_goals

C:\Users\simod\AppData\Local\Temp\ipykernel_6772\3946588157.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elif(df_team_goals['stage_name'][ind] == 'final'): df_team_goals['stage_number'][ind] = 7
C:\Users\simod\AppData\Local\Temp\ipykernel_6772\3946588157.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if(df_team_goals['stage_name'][ind] == 'group stage'): df_team_goals['stage_number'][ind] = 0
C:\Users\simod\AppData\Local\Temp\ipykernel_6772\3946588157.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydat

,tournament_id,team_name,goals,stage_name,stage_number
0,WC-1930,Argentina,18,final,7
1,WC-1930,Belgium,0,group stage,0
2,WC-1930,Bolivia,0,group stage,0
3,WC-1930,Brazil,5,group stage,0
4,WC-1930,Chile,5,group stage,0
...,...,...,...,...,...
452,WC-2018,Spain,7,round of 16,2
453,WC-2018,Sweden,6,quarter-finals,3
454,WC-2018,Switzerland,5,round of 16,2
455,WC-2018,Tunisia,5,group stage,0


In [32]:
df_matches = pd.read_csv(PATH_TO_WRITE_ETL + "matches.csv")
df_team_goals = pd.read_csv('./etl_Simone/team_goals.csv')

df_matches = df_matches[['tournament_id', 'match_id', 'home_team_name', 'away_team_name', 'home_team_score', 'away_team_score']]
df_home_subiti = df_matches[['tournament_id', 'home_team_name', 'away_team_score']]
df_away_subiti = df_matches[['tournament_id', 'away_team_name', 'home_team_score']]
df_home_subiti = df_home_subiti.groupby(['tournament_id', 'home_team_name'])['away_team_score'].sum().reset_index(name = 'goal_subiti')
df_away_subiti = df_away_subiti.groupby(['tournament_id', 'away_team_name'])['home_team_score'].sum().reset_index(name = 'goal_subiti')
df_home_subiti.rename(columns = {'home_team_name':'team_name'}, inplace = True)
df_away_subiti.rename(columns = {'away_team_name':'team_name'}, inplace = True)
dataframes = [df_home_subiti, df_away_subiti]
result = pd.concat(dataframes).reset_index()
result.drop(['index'], axis=1, inplace=True)
result = result.groupby(['tournament_id', 'team_name'])['goal_subiti'].sum().reset_index(name = 'goal_subiti')
df_team_goals['goal_subiti'] = result['goal_subiti']
df_team_goals['year'] = 0

for i in df_team_goals.index:
    if(df_team_goals['tournament_id'][i] == 'WC-1930'): df_team_goals['year'][i] = 1930
    elif(df_team_goals['tournament_id'][i] == 'WC-1934'): df_team_goals['year'][i] = 1934
    elif(df_team_goals['tournament_id'][i] == 'WC-1938'): df_team_goals['year'][i] = 1938
    elif(df_team_goals['tournament_id'][i] == 'WC-1950'): df_team_goals['year'][i] = 1950
    elif(df_team_goals['tournament_id'][i] == 'WC-1954'): df_team_goals['year'][i] = 1954
    elif(df_team_goals['tournament_id'][i] == 'WC-1958'): df_team_goals['year'][i] = 1958
    elif(df_team_goals['tournament_id'][i] == 'WC-1962'): df_team_goals['year'][i] = 1962
    elif(df_team_goals['tournament_id'][i] == 'WC-1966'): df_team_goals['year'][i] = 1966
    elif(df_team_goals['tournament_id'][i] == 'WC-1970'): df_team_goals['year'][i] = 1970
    elif(df_team_goals['tournament_id'][i] == 'WC-1974'): df_team_goals['year'][i] = 1974
    elif(df_team_goals['tournament_id'][i] == 'WC-1978'): df_team_goals['year'][i] = 1978
    elif(df_team_goals['tournament_id'][i] == 'WC-1982'): df_team_goals['year'][i] = 1982
    elif(df_team_goals['tournament_id'][i] == 'WC-1986'): df_team_goals['year'][i] = 1986
    elif(df_team_goals['tournament_id'][i] == 'WC-1990'): df_team_goals['year'][i] = 1990
    elif(df_team_goals['tournament_id'][i] == 'WC-1994'): df_team_goals['year'][i] = 1994
    elif(df_team_goals['tournament_id'][i] == 'WC-1998'): df_team_goals['year'][i] = 1998
    elif(df_team_goals['tournament_id'][i] == 'WC-2002'): df_team_goals['year'][i] = 2002
    elif(df_team_goals['tournament_id'][i] == 'WC-2006'): df_team_goals['year'][i] = 2006
    elif(df_team_goals['tournament_id'][i] == 'WC-2010'): df_team_goals['year'][i] = 2010
    elif(df_team_goals['tournament_id'][i] == 'WC-2014'): df_team_goals['year'][i] = 2014
    elif(df_team_goals['tournament_id'][i] == 'WC-2018'): df_team_goals['year'][i] = 2018

df_team_goals = df_team_goals[['tournament_id', 'year', 'team_name', 'goals', 'goal_subiti', 'stage_name', 'stage_number']]

df_team_goals.to_csv('./etl_Simone/team_goals.csv', index = False)

df_team_goals

C:\Users\simod\AppData\Local\Temp\ipykernel_3200\1401451689.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if(df_team_goals['tournament_id'][i] == 'WC-1930'): df_team_goals['year'][i] = 1930
C:\Users\simod\AppData\Local\Temp\ipykernel_3200\1401451689.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elif(df_team_goals['tournament_id'][i] == 'WC-1934'): df_team_goals['year'][i] = 1934
C:\Users\simod\AppData\Local\Temp\ipykernel_3200\1401451689.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-

,tournament_id,year,team_name,goals,goal_subiti,stage_name,stage_number
0,WC-1930,1930,Argentina,18,9,final,7
1,WC-1930,1930,Belgium,0,4,group stage,0
2,WC-1930,1930,Bolivia,0,8,group stage,0
3,WC-1930,1930,Brazil,5,2,group stage,0
4,WC-1930,1930,Chile,5,3,group stage,0
...,...,...,...,...,...,...,...
452,WC-2018,2018,Spain,7,6,round of 16,2
453,WC-2018,2018,Sweden,6,4,quarter-finals,3
454,WC-2018,2018,Switzerland,5,5,round of 16,2
455,WC-2018,2018,Tunisia,5,8,group stage,0


In [18]:
df_team_goals = pd.read_csv('./etl_Simone/team_goals.csv')
df_bookings = pd.read_csv(PATH_TO_WRITE_ETL + 'bookings.csv')
df_bookings['card'] = df_bookings['yellow_card'] + df_bookings['red_card']
df_bookings = df_bookings.groupby(['tournament_id', 'team_name'])['card'].sum().reset_index(name = 'cards')
df_team_goals['cards'] = 0

for i in df_bookings.index:
    for j in df_team_goals.index:
        if(df_bookings['tournament_id'][i] == df_team_goals['tournament_id'][j] and df_bookings['team_name'][i] == df_team_goals['team_name'][j]):
            df_team_goals['cards'][j] = df_bookings['cards'][i]

df_team_goals.to_csv('./etl_Simone/team_goals.csv', index = False)

df_team_goals

C:\Users\simod\AppData\Local\Temp\ipykernel_12084\1776517944.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_team_goals['cards'][j] = df_bookings['cards'][i]


,tournament_id,year,team_name,goals,goal_subiti,stage_name,stage_number,cards
0,WC-1930,1930,Argentina,18,9,final,7,0
1,WC-1930,1930,Belgium,0,4,group stage,0,0
2,WC-1930,1930,Bolivia,0,8,group stage,0,0
3,WC-1930,1930,Brazil,5,2,group stage,0,0
4,WC-1930,1930,Chile,5,3,group stage,0,0
...,...,...,...,...,...,...,...,...
452,WC-2018,2018,Spain,7,6,round of 16,2,2
453,WC-2018,2018,Sweden,6,4,quarter-finals,3,8
454,WC-2018,2018,Switzerland,5,5,round of 16,2,10
455,WC-2018,2018,Tunisia,5,8,group stage,0,4
